# 実装

## 準備

In [ ]:
# ライブラリのインポート
from typing import Optional, Tuple
import random
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib
import pygrib
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from shapely.errors import ShapelyDeprecationWarning

warnings.filterwarnings('ignore', category=ShapelyDeprecationWarning)

plt.rcParams["font.size"] = 18  # 図の文字サイズを大きくしておく
seed = 42  # 乱数状態の固定

In [ ]:
# 各種設定
path = "./anl_p125_hgt.2012011012"  # JRA-55データのパス
level = 850  # 気圧面
projection = ccrs.PlateCarree()  # 正距円筒図法

## データの読み込み

In [ ]:
def inquire_grib_data(path: str) -> None:
    """データ概要を表示

    Args:
        path(str): 読むデータのパス 
    """
    grbs = pygrib.open(path)
    for grb in grbs:
        print(grb)
    return

In [ ]:
inquire_grib_data(path)

In [ ]:
def read_grib_data(
    path: str,
    name: Optional[str] = None,
    level: Optional[int] = None
)-> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """データを読む

    - levelを与えないと全３次元データ
    
    Args:
        path(str): 読むデータのパス
        name(Optional[str]): 変数名(anl_surf125に対して与える)
        level(Optional[int]): 気圧面（anl_p125に対して与える）

    Returns:
        Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]: 経度, 緯度, 気圧面, データ
    """
    grbs = pygrib.open(path)

    if name is not None:
        alines = grbs.select(name=name)
    elif level is not None:
        alines = grbs.select(level=level)
    else:
        alines = grbs.select()

    lat, lon = alines[0].latlons()  # lonは経度、latは緯度データ: (ny,nx)の２次元格子です
    ny, nx = lat.shape
    nline = len(alines)
    gdata = np.empty((nline, ny, nx), dtype="float32")
    levels = np.empty((nline), dtype="float32")

    for iline, aline in enumerate(alines):
        gdata[iline, :, :] = aline.values[::-1, :]
        levels[iline] = aline["level"]

    return lon, lat[::-1], levels, gdata

In [ ]:
lon, lat, levels, data = read_grib_data(path, level=level)
print(lon, lon.shape, lat, lat.shape, levels, data, data.shape, sep="\n\n")

In [ ]:
data850 = data[0]
data850.shape

## 等高線を描画

In [ ]:
# 描画領域の設定
fig = plt.figure(figsize=(20, 15))
ax = fig.add_subplot(projection=projection)

# 地図関係
ax.add_feature(cfeature.LAND)
ax.coastlines(lw=0.5)
ax.gridlines(linestyle='-', color='gray', draw_labels=True)

# 等高線を描画
# オプションはこちらを参照のこと：https://matplotlib.org/3.5.1/api/_as_gen/matplotlib.pyplot.contour.html
cont = ax.contour(lon, lat, data850, transform=ccrs.PlateCarree(), cmap="rainbow")
cont.clabel(fmt='%1.1f', fontsize=14)

![figの構造](https://cdn-ak.f.st-hatena.com/images/fotolife/Y/YutaKa/20210115/20210115154737.png)

## 図法の変更・塗りつぶし 2 次元等高線図

In [ ]:
projection_lambert = ccrs.LambertConformal()  # ランベルト正角円錐図法

In [ ]:
fig = plt.figure(figsize=(20, 15))
ax = fig.add_subplot(projection=projection_lambert)

ax.add_feature(cfeature.LAND)
ax.coastlines(lw=0.5)
ax.gridlines(linestyle='-', color='gray', draw_labels=True)
ax.set_extent([120, 150, 20, 50], ccrs.PlateCarree())  # 範囲の指定は正距円筒図法を利用

contf = ax.contourf(lon, lat, data850, transform=ccrs.PlateCarree(), cmap="rainbow",
                    extend='both', levels=range(1200, 1600, 50))
fig.colorbar(contf, orientation='vertical', shrink=0.8)

## 緯度経度データの描画

In [ ]:
df_weather = pd.read_html("https://www.geekpage.jp/web/livedoor-weather-hacks/latlng.php",
                            header=0)[0]
# ダミーデータの作成（コードは気にしなくて大丈夫です）
# 適当なページからとってきたので、一部間違っていて地図からはみ出します

df_weather = df_weather.dropna(axis=0)
df_weather["天気"] = [random.choice(["晴れ", "曇り", "雨"]) for i in range(len(df_weather))]
df_weather

In [ ]:
fig = plt.figure(figsize=(20, 15))
ax = fig.add_subplot(projection=projection_lambert)

ax.add_feature(cfeature.LAND)
ax.coastlines(lw=0.5)
ax.gridlines(linestyle='-', color='gray', draw_labels=True)
ax.set_extent([120, 150, 20, 50], ccrs.PlateCarree())

cont = ax.contour(lon, lat, data850, transform=ccrs.PlateCarree(), cmap="rainbow",
                   levels=range(1200, 1600, 50))
cont.clabel(fmt='%1.1f', fontsize=14)

# 散布図を描画
# オプションはこちらを参照のこと：https://seaborn.pydata.org/generated/seaborn.scatterplot.html
# sns(seaborn)はmatplotlibのラッパーライブラリでpd(pandas)のDataFrameなどとの相性が非常に良い
sns.scatterplot(x="経度(lng)", y="緯度(lat)", hue="天気", data=df_weather,
                transform=ccrs.PlateCarree(), palette="rainbow", ax=ax)

## クラスター分析の結果を表示

In [ ]:
df_cluster = pd.read_csv("cluster_data.csv", encoding="shift-jis")
df_cluster

In [ ]:
fig = plt.figure(figsize=(20, 15))
ax = fig.add_subplot(projection=projection)

ax.coastlines(lw=0.5, color="gray")
# メモリだけ表示するのはめんどくさそうだったので、幅0の緯線経線を引いて力技で表示
ax.gridlines(draw_labels=True, linewidth=0)
ax.set_extent([120, 150, 20, 50], ccrs.PlateCarree())

# styleでどの値に応じてマーカーの形を変えるかを規定して、markersで実際に利用するマーカーの形を記述する
sns.scatterplot(x="lon", y="lat", hue="cluster_NO", style="frag", markers=["o", "^"],
                data=df_cluster, s=100, palette="rainbow_r", ax=ax)

In [ ]:
# ちなみにpythonでクラスター分析(k-means法)
from sklearn.cluster import KMeans

k_means = KMeans(n_clusters=6, random_state=seed).fit(df_cluster.loc[:, "1":"12"])
df_cluster_copy = df_cluster.copy()
df_cluster_copy["cluster_py"] = k_means.labels_
df_cluster_copy

In [ ]:
fig = plt.figure(figsize=(20, 15))
ax = fig.add_subplot(projection=projection)

ax.coastlines(lw=0.5, color="gray")
ax.gridlines(draw_labels=True, linewidth=0)
ax.set_extent([120, 150, 20, 50], ccrs.PlateCarree())

sns.scatterplot(x="lon", y="lat", hue="cluster_py", style="frag", markers=["o", "^"],
                data=df_cluster_copy, s=100, palette="rainbow_r", ax=ax)

In [ ]:
from sklearn.decomposition import PCA

x_pca = pd.DataFrame(PCA(n_components=2).fit_transform(df_cluster.loc[:, "1":"12"]),
                       columns=["1st", "2nd"])
x_pca["cluster"] = df_cluster_copy["cluster_py"]

# 実は描画領域の設定をしなくても使える
sns.scatterplot(x="1st", y="2nd", hue="cluster", data=x_pca, palette="rainbow_r")
plt.legend(bbox_to_anchor=(1.05, 1))

# 参考文献

- [気象データの出入力](https://humet.sci.hokudai.ac.jp/~meteo/tutorial.html)
- [pygrib公式ドキュメント](https://jswhit.github.io/pygrib/api.html#module-pygrib)
- [GPVでのpygrib利用例](https://qiita.com/kurukuruz/items/6fc0be9efa34a2fd6741)
- [cartopy公式ドキュメント（図法一覧）](https://scitools.org.uk/cartopy/docs/latest/reference/projections.html)
- [cartopyまとめ](https://yyousuke.github.io/matplotlib/cartopy.html)
- [図法関連（日本語記事）](https://metpost.hatenablog.com/entry/2015/11/05/180006)
- [matplotlib公式ドキュメント(contour)](https://matplotlib.org/3.5.1/api/_as_gen/matplotlib.pyplot.contour.html)
- [seaborn公式ドキュメント(scatterplot)](https://seaborn.pydata.org/generated/seaborn.scatterplot.html)

In [ ]:
# このセルはこのままでは動かないです

# shp形式の地図データを読み込む
# 地図データのダウンロード
# https://www.naturalearthdata.com/downloads/ (Physical -> Coastline)
import cartopy.io.shapereader as shpreader

shp_path = "shpファイルのパス"
plt.figure(figsize=(20, 15))
ax = plt.axes(projection=ccrs.PlateCarree())
shapes = list(shpreader.Reader(shp_path).geometries())
ax.add_geometries(shapes, ccrs.PlateCarree(), edgecolor='black', facecolor='gray', alpha=0.3)
cont = plt.contour(lon, lat, data[0])
cont.clabel(fmt='%1.1f', fontsize=14)
plt.show()